In [1]:
import torch
import torch.utils.data.dataloader
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.autograd import Variable
import time
from MCNN import MCNN

# Settings
batch_size = 10000
test_batch_size = 10000
seed = 1
torch.manual_seed(seed)

# Dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0,), (1,))
                   ])),
    batch_size=batch_size, shuffle=True)


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0,), (1,))
                   ])),
    batch_size=batch_size, shuffle=True)

# Model 
model = MCNN()
params = list(model.parameters())

init=time.time()

for batch_idx, (data, target) in enumerate(train_loader):
    data, target = Variable(data), Variable(target)
    x = model.forwardConvTraining(data, params,1) #optimizer removed at the end
    output = model.forwardLastLayer(x, target)

ending=time.time()
print("Training time: " + str(ending-init))

# Train accuracy
def train_accuracy():
    correct = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        output = model.forward(data)
        pred = output.data.max(1)[1]
        correct += pred.eq(target.data).cpu().sum()
    
    print('\nTrain set accuracy: {}/{} ({:.2f}%)\n'.format(
        correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))

# Test accuracy
def test_accuracy():
    correct = 0

    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model.forward(data)
        pred=output.data.max(1)[1]
        correct += pred.eq(target.data).cpu().sum()
        
    print('\nAccuracy: {}/{} ({:.2f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

test_accuracy()

C:\Users\marth\MCNN\MoorePenrose.py:17: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.M=Variable(torch.inverse(self.C*torch.eye(self.dimInput)),requires_grad=False, volatile=True)
C:\Users\marth\MCNN\MoorePenrose.py:23: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  I = Variable(torch.eye(dimInput),requires_grad=False, volatile=True)
C:\Users\marth\MCNN\MoorePenrose.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.M = Variable(torch.inverse(xtx.data + self.C * (I.data+self.L*S.data)),requires_grad=False, volatile=True)
C:\Users\marth\MCNN\MoorePenrose.py:70: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  oneHotTarget=Variable(oneHotTarget,requires_grad=False, volatile=True)


Training time: 1351.0555272102356


C:\Users\marth\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Accuracy: 9708/10000 (97.08%)

